# Clasificador de flores

Bienvenidos. En este ejercicio haremos un clasificador para flores. Usaremos el siguiente dataset: http://www.robots.ox.ac.uk/~vgg/data/flowers/, que contiene 102 categorías y 8192 imágenes de flores de alta calidad. En un momento te diré cómo descargar el dataset ya editado por mi para hacerlo más fácil de manejar.

Al final lograremos llegar a 99.21% de accuracy, aún cuando tenemos 102 tipos de flores diferentes. Esto es muy impresionante, sobre todo considerando que en [este paper](https://www.robots.ox.ac.uk/~vgg/research/flowers_demo/docs/Chai11.pdf) reportan 81% de accuracy, y que en [este otro](http://www.cs.huji.ac.il/~daphna/IsraeliFlowers/flower_classification.html) reportan 93.8%! 

Es decir, lo haremos **7.7 veces mejor** que ese último!! (0.8% vs 6.2% de error)

Vamos a incluir las bibliotecas que usaremos, que son [fast.ai](https://www.fast.ai/) y [pytorch](https://pytorch.org).

In [10]:
import fastai.vision.all as fv

from pathlib import Path

### ¿Cómo son nuestros datos?

Siempre antes de empezar a trabajar en algún problema, hay que entender bien qué problema queremos resolver. En este caso, simplemente queremos clasificar flores en base a sus imágenes.

Después, hay que entender los [datos](https://my.pcloud.com/publink/show?code=XZiwtw7Z15TY80dPm4b3KwmNcf1ChfphgM5y). En este caso, de las flores, pensemos que ya los descargamos, descomprimimos y están en un carpeta llamada "flowers".

In [6]:
!ls flowers/

models/  tmp/  train/  valid/


Posiblemente tú no tendrás tmp ni models aún. Eso se crean solitos después.

In [7]:
!ls flowers/train

alpine-sea-holly/    fire-lily/                  peruvian-lily/
anthurium/           foxglove/                   petunia/
artichoke/           frangipani/                 pincushion-flower/
azalea/              fritillary/                 pink-primrose/
ball-moss/           garden-phlox/               pink-yellow-dahlia/
balloon-flower/      gaura/                      poinsettia/
barbeton-daisy/      gazania/                    primula/
bearded-iris/        geranium/                   prince-of-wales-feathers/
bee-balm/            giant-white-arum-lily/      purple-coneflower/
bird-of-paradise/    globe-flower/               red-ginger/
bishop-of-llandaff/  globe-thistle/              rose/
blackberry-lily/     grape-hyacinth/             ruby-lipped-cattleya/
black-eyed-susan/    great-masterwort/           siam-tulip/
blanket-flower/      hard-leaved-pocket-orchid/  silverbush/
bolero-deep-blue/    hibiscus/                   snapdragon/
bougainvillea/       hippeastrum/            

Ya entendimos cómo están las imágenes y su clasificación. (Nota: el archivo original que bajas de oxford no estaba bien acomodado así, estaba en un csv y así, medio feito, yo lo convertí.)

In [11]:
flowers = Path("flowers/")

In [12]:
(flowers/"train").ls()

(#102) [Path('flowers/train/stemless-gentian'),Path('flowers/train/rose'),Path('flowers/train/cautleya-spicata'),Path('flowers/train/monkshood'),Path('flowers/train/globe-flower'),Path('flowers/train/tiger-lily'),Path('flowers/train/peruvian-lily'),Path('flowers/train/bishop-of-llandaff'),Path('flowers/train/azalea'),Path('flowers/train/yellow-iris')...]

In [14]:
files = fv.get_image_files("flowers"); files

(#8188) [Path('flowers/train/stemless-gentian/image_05227.jpg'),Path('flowers/train/stemless-gentian/image_05216.jpg'),Path('flowers/train/stemless-gentian/image_05253.jpg'),Path('flowers/train/stemless-gentian/image_05269.jpg'),Path('flowers/train/stemless-gentian/image_05214.jpg'),Path('flowers/train/stemless-gentian/image_05273.jpg'),Path('flowers/train/stemless-gentian/image_05248.jpg'),Path('flowers/train/stemless-gentian/image_05261.jpg'),Path('flowers/train/stemless-gentian/image_05243.jpg'),Path('flowers/train/stemless-gentian/image_05263.jpg')...]

In [19]:
fv.doc(fv.get_image_files)

get_image_files [source] get_image_files ( path , recurse = True , folders = None ) 
 
 Get image files in path recursively, only in folders , if specified. 
 Show in docs

## Abriendo los datos en fastai

Dado que nuestros datos están bien acomodados en folders, vamos a crear un "ImageDataBunch" a partir del folder.

Usaremos "data augmentation" (más sobre esto después), estableciendo transformadas.

In [18]:
fv.doc(fv.aug_transforms)

aug_transforms [source] aug_transforms ( mult = 1.0 , do_flip = True , flip_vert = False , max_rotate = 10.0 , min_zoom = 1.0 , max_zoom = 1.1 , max_lighting = 0.2 , max_warp = 0.2 , p_affine = 0.75 , p_lighting = 0.75 , xtra_tfms = None , size = None , mode = 'bilinear' , pad_mode = 'reflection' , align_corners = True , batch = False , min_scale = 1.0 ) 
 
 Utility func to easily create a list of flip, rotate, zoom, warp, lighting transforms. 
 Show in docs

**Nota:** bs = batch size. Es decir, cuántas imágenes pasa al mismo tiempo a la red neuronal. Mi tarjeta de video tiene 11GB de memoria, por es puedo poner un número relativamente grande. Si obtienes "CUDA error: out of memory", disminuye la batch size!

Podríamos también cargar los datos "a mano", como se muestra en las siguientes celdas. Para más información, ver el [datablock-api](https://docs.fast.ai/data_block.html)

In [ ]:
def load_data(folder, img_size, batch_size):
    tfms = fv.get_transforms(flip_vert=True, max_rotate=360, max_lighting=0.3,max_zoom=1.2,max_warp=0.2) #tfms = transforms
    data = (fv.ImageList
            .from_folder(folder)
            .split_by_folder()
            .label_from_folder()
            .transform(tfms,size=img_size)
            .databunch(bs=batch_size))
    return data

In [ ]:
data = load_data(flowers, img_size=224, batch_size=64)

In [ ]:
data.show_batch(rows=3)

## Entrenamiento

Ahora crearemos un objeto de tipo "Learner" (más sobre esto después), con arquitectura de "resnet18". Qué es esto? No te preocupes!

In [ ]:
learner = fv.cnn_learner(data, fv.models.resnet18, metrics=fai.accuracy)

### Entrenamos *sólo de las últimas capas*

Vamos a usar la función "fit_one_cycle", que es una manera muy rápida de entrenar (discutiremos por qué después). wd significa "weight decay" y es para hacer "regularización l2" (veremos qué significa después)

In [ ]:
learner.fit_one_cycle(4)

In [ ]:
learner.recorder.plot_lr(show_moms=True)

No sabemos qué taza de aprendizaje usar, y de hecho hemos estado usando la que trae por defecto (0.003). ¿Qué learning rate usamos? Pues vamos a averiguar!

In [ ]:
learner.lr_find(); learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(4, max_lr = 1e-2)

In [ ]:
learner.lr_find(); learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(3,1e-4)

In [ ]:
learner.save('stage1')

In [ ]:
learner.load("stage1");

### Taza de aprendizaje cíclica 

Como funciona el método fit_one_cycle? Se ha encontrado, experimentalmente, que ir modificando la taza de aprendizaje (learning rate) en "ciclos" mejora mucho el resultado final de la red. Después entenderemos la razón, pero tiene que ver con que son mejores los "mínimos gordos" que los "mínimos flacos". [Paper](https://arxiv.org/abs/1506.01186)

In [ ]:
learner.recorder.plot_losses()

In [ ]:
learner.recorder.plot_lr(show_moms=True)

In [ ]:
learner.model

In [ ]:
learner.summary()

### Entrenamiento de TODAS las capas

En realidad **no hemos estado entrenando a toda la red neuronal!!** Sólo entrenamos las últimas dos capas, y las demás estaban **pre-entrenadas en imagenet**. Por defecto, si pides una arquitectura pre-hecha, vienen "congeladas" las primeras capas. Vamos a entrenar con cuidado toda la red.

In [ ]:
learner.unfreeze() # ahora hay que tener cuidado!

In [ ]:
learner.lr_find(); learner.recorder.plot()

In [ ]:
learner.summary()

Vamos a entrenar toda la red, pero vamos a modificar las primeras capas **menos que las últimas**. Para entender por qué, veamos este paper: [Visualizing and Understading ...](https://cs.nyu.edu/~fergus/papers/zeilerECCV2014.pdf)

In [ ]:
learner.fit_one_cycle(5, max_lr=slice(1e-5,1e-3))

In [ ]:
learner.save('stage2')

In [ ]:
learner.show_results(ds_type=fai.DatasetType.Train, rows=3) # Podemos cambiar Valid por Train

# Interpretando los resultados

#### Test Time Augmentation
Así como hicimos data augmentation al entrenar, ¿por qué no hacerlo también al inferir?

In [ ]:
fai.accuracy(*learner.TTA())

Vamos a revisar los resultados que tenemos

In [ ]:
interp = learner.interpret()

In [ ]:
interp.plot_top_losses(9,heatmap=False)

Las dos que predecimos mallow pero deberíamos predecir camellia... están de hecho mal clasificadas en el validation set!

Esto es algo muy común, los datasets no son perfectos. Y hacer este tipo de cosas nos da herramientas para encontrar estos errores.

In [ ]:
interp.plot_confusion_matrix(figsize=(16,16), dpi=300)

In [ ]:
interp.most_confused(1)